# 제품 한개만 추천해줌 -> top 3 추천으로 변경
# 로고 이미지 반영 X
# 피부 타입, 원하는 카테고리, 피부 고민 순차적으로 선택하게끔 변경
# 사용자 정보(이전 정보) 저장
# 대화만으로도 질문&대답할 수 있게 디버깅.

#

In [ ]:
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>INGREVIA - 성분 기반 맞춤 화장품 추천</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }
        
        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica Neue', Arial, sans-serif;
            background: linear-gradient(135deg, #f8fdf8 0%, #f0f8f0 50%, #e8f5e8 100%);
            min-height: 100vh;
            display: flex;
            justify-content: center;
            align-items: center;
            padding: 20px;
        }
        
        .chat-container {
            max-width: 900px;
            width: 100%;
            height: 90vh;
            background: rgba(255, 255, 255, 0.95);
            backdrop-filter: blur(10px);
            border-radius: 24px;
            box-shadow: 0 20px 60px rgba(76, 114, 89, 0.1), 0 8px 20px rgba(76, 114, 89, 0.08);
            border: 1px solid rgba(76, 114, 89, 0.1);
            display: flex;
            flex-direction: column;
            overflow: hidden;
        }
        
        .header {
            background: linear-gradient(135deg, rgba(76, 114, 89, 0.9) 0%, rgba(93, 138, 107, 0.9) 100%);
            color: white;
            padding: 25px;
            text-align: center;
            position: relative;
            backdrop-filter: blur(10px);
        }
        
        .logo {
            width: 70px;
            height: 70px;
            margin: 0 auto 15px;
            background: rgba(255, 255, 255, 0.95);
            border-radius: 50%;
            display: flex;
            align-items: center;
            justify-content: center;
            box-shadow: 0 4px 15px rgba(0, 0, 0, 0.1);
            position: relative;
            overflow: hidden;
        }
        
        .logo img {
            width: 60px;
            height: 60px;
            border-radius: 50%;
            object-fit: cover;
        }
        
        .logo-fallback {
            display: none;
        }
        
        .logo svg {
            width: 35px;
            height: 35px;
        }
        
        .header h1 {
            font-size: 28px;
            font-weight: 300;
            letter-spacing: 4px;
            margin-bottom: 8px;
            text-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }
        
        .header p {
            font-size: 15px;
            opacity: 0.9;
            font-weight: 300;
        }
        
        .chat-messages {
            flex: 1;
            padding: 25px;
            overflow-y: auto;
            display: flex;
            flex-direction: column;
            gap: 18px;
        }
        
        .message {
            max-width: 85%;
            padding: 15px 20px;
            border-radius: 20px;
            font-size: 14px;
            line-height: 1.6;
            animation: fadeIn 0.4s ease-out;
        }
        
        .message.bot {
            background: linear-gradient(135deg, rgba(248, 253, 248, 0.8) 0%, rgba(240, 248, 240, 0.9) 100%);
            border: 1px solid rgba(76, 114, 89, 0.15);
            align-self: flex-start;
            border-bottom-left-radius: 8px;
            backdrop-filter: blur(5px);
        }
        
        .message.user {
            background: linear-gradient(135deg, rgba(76, 114, 89, 0.85) 0%, rgba(93, 138, 107, 0.85) 100%);
            color: white;
            align-self: flex-end;
            border-bottom-right-radius: 8px;
            backdrop-filter: blur(5px);
        }
        
        .selection-panel {
            background: rgba(248, 253, 248, 0.6);
            border: 1.5px solid rgba(76, 114, 89, 0.15);
            border-radius: 16px;
            padding: 20px;
            margin: 12px 0;
            backdrop-filter: blur(5px);
            transition: all 0.3s ease;
        }
        
        .selection-panel.completed {
            opacity: 0.6;
            pointer-events: none;
        }
        
        .selection-panel.current {
            border-color: rgba(76, 114, 89, 0.4);
            background: rgba(248, 253, 248, 0.9);
            box-shadow: 0 4px 12px rgba(76, 114, 89, 0.1);
        }
        
        .selection-title {
            font-weight: 600;
            color: rgba(76, 114, 89, 0.9);
            margin-bottom: 15px;
            font-size: 16px;
        }
        
        .button-group {
            display: flex;
            flex-wrap: wrap;
            gap: 10px;
        }
        
        .selection-btn {
            background: rgba(255, 255, 255, 0.8);
            border: 1.5px solid rgba(76, 114, 89, 0.25);
            color: rgba(76, 114, 89, 0.85);
            padding: 10px 18px;
            border-radius: 25px;
            cursor: pointer;
            transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1);
            font-size: 14px;
            font-weight: 500;
            backdrop-filter: blur(5px);
        }
        
        .selection-btn:hover {
            background: rgba(240, 248, 240, 0.9);
            border-color: rgba(76, 114, 89, 0.4);
            transform: translateY(-2px);
            box-shadow: 0 4px 12px rgba(76, 114, 89, 0.15);
        }
        
        .selection-btn.selected {
            background: linear-gradient(135deg, rgba(76, 114, 89, 0.9) 0%, rgba(93, 138, 107, 0.9) 100%);
            color: white;
            border-color: rgba(76, 114, 89, 0.6);
            transform: translateY(-1px);
            box-shadow: 0 6px 18px rgba(76, 114, 89, 0.25);
        }
        
        .product-gallery {
            margin: 15px 0;
        }
        
        .product-scroll {
            display: flex;
            overflow-x: auto;
            gap: 15px;
            padding: 10px 0;
            scroll-snap-type: x mandatory;
        }
        
        .product-scroll::-webkit-scrollbar {
            height: 6px;
        }
        
        .product-scroll::-webkit-scrollbar-track {
            background: rgba(76, 114, 89, 0.1);
            border-radius: 3px;
        }
        
        .product-scroll::-webkit-scrollbar-thumb {
            background: rgba(76, 114, 89, 0.3);
            border-radius: 3px;
        }
        
        .product-card {
            background: rgba(255, 255, 255, 0.9);
            border: 1.5px solid rgba(76, 114, 89, 0.1);
            border-radius: 16px;
            padding: 15px;
            min-width: 280px;
            backdrop-filter: blur(5px);
            box-shadow: 0 4px 15px rgba(76, 114, 89, 0.08);
            transition: all 0.3s ease;
            scroll-snap-align: start;
            cursor: pointer;
        }
        
        .product-card:hover {
            transform: translateY(-5px);
            box-shadow: 0 8px 25px rgba(76, 114, 89, 0.15);
        }
        
        .product-image {
            width: 100%;
            height: 120px;
            background: linear-gradient(135deg, #f8fdf8, #e8f5e8);
            border-radius: 12px;
            display: flex;
            align-items: center;
            justify-content: center;
            margin-bottom: 12px;
            position: relative;
            overflow: hidden;
        }
        
        .product-image img {
            width: 80px;
            height: 80px;
            object-fit: cover;
            border-radius: 8px;
        }
        
        .product-info {
            text-align: center;
        }
        
        .product-rank {
            background: linear-gradient(135deg, rgba(76, 114, 89, 0.1) 0%, rgba(93, 138, 107, 0.1) 100%);
            color: rgba(76, 114, 89, 0.8);
            padding: 4px 8px;
            border-radius: 12px;
            font-size: 11px;
            font-weight: 600;
            margin-bottom: 8px;
            display: inline-block;
        }
        
        .product-name {
            font-weight: 600;
            color: rgba(76, 114, 89, 0.9);
            font-size: 14px;
            margin-bottom: 4px;
        }
        
        .product-brand {
            font-size: 12px;
            color: rgba(76, 114, 89, 0.7);
            margin-bottom: 8px;
        }
        
        .product-price {
            font-weight: 600;
            color: rgba(76, 114, 89, 0.9);
            font-size: 13px;
            margin-bottom: 2px;
        }
        
        .product-volume {
            font-size: 11px;
            color: rgba(76, 114, 89, 0.6);
        }
        
        .input-container {
            padding: 25px;
            background: rgba(255, 255, 255, 0.95);
            border-top: 1px solid rgba(76, 114, 89, 0.1);
            backdrop-filter: blur(10px);
        }
        
        .input-wrapper {
            display: flex;
            gap: 12px;
            align-items: flex-end;
        }
        
        .chat-input {
            flex: 1;
            padding: 15px 20px;
            border: 1.5px solid rgba(76, 114, 89, 0.25);
            border-radius: 25px;
            resize: none;
            font-family: inherit;
            font-size: 14px;
            outline: none;
            transition: all 0.3s ease;
            max-height: 120px;
            background: rgba(248, 253, 248, 0.5);
            backdrop-filter: blur(5px);
        }
        
        .chat-input:focus {
            border-color: rgba(76, 114, 89, 0.5);
            background: rgba(248, 253, 248, 0.8);
            box-shadow: 0 0 0 3px rgba(76, 114, 89, 0.1);
        }
        
        .send-btn {
            width: 50px;
            height: 50px;
            background: linear-gradient(135deg, rgba(76, 114, 89, 0.9) 0%, rgba(93, 138, 107, 0.9) 100%);
            color: white;
            border: none;
            border-radius: 50%;
            cursor: pointer;
            display: flex;
            align-items: center;
            justify-content: center;
            transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1);
            backdrop-filter: blur(5px);
            box-shadow: 0 4px 15px rgba(76, 114, 89, 0.2);
        }
        
        .send-btn:hover {
            transform: scale(1.05);
            box-shadow: 0 6px 20px rgba(76, 114, 89, 0.3);
        }
        
        .send-btn:active {
            transform: scale(0.95);
        }
        
        .welcome-message {
            text-align: center;
            padding: 40px 20px;
            color: rgba(76, 114, 89, 0.8);
        }
        
        .welcome-message h2 {
            font-size: 22px;
            font-weight: 400;
            margin-bottom: 12px;
        }
        
        .welcome-message p {
            font-size: 15px;
            opacity: 0.8;
            line-height: 1.6;
        }
        
        .quick-start {
            background: rgba(248, 253, 248, 0.6);
            border: 1.5px solid rgba(76, 114, 89, 0.15);
            border-radius: 16px;
            padding: 25px;
            margin: 25px 0;
            backdrop-filter: blur(5px);
        }
        
        .quick-start h3 {
            color: rgba(76, 114, 89, 0.9);
            font-size: 18px;
            margin-bottom: 20px;
            font-weight: 500;
        }
        
        .typing-indicator {
            display: none;
            align-items: center;
            gap: 8px;
            color: rgba(76, 114, 89, 0.6);
            font-style: italic;
            font-size: 13px;
            padding: 12px 16px;
        }
        
        .typing-dots {
            display: flex;
            gap: 3px;
        }
        
        .typing-dot {
            width: 5px;
            height: 5px;
            background: rgba(76, 114, 89, 0.6);
            border-radius: 50%;
            animation: typing 1.4s infinite ease-in-out;
        }
        
        .typing-dot:nth-child(2) { animation-delay: 0.2s; }
        .typing-dot:nth-child(3) { animation-delay: 0.4s; }
        
        .progress-indicator {
            display: none;
            text-align: center;
            padding: 20px;
            color: rgba(76, 114, 89, 0.7);
            font-size: 14px;
        }
        
        .progress-bar {
            width: 100%;
            height: 4px;
            background: rgba(76, 114, 89, 0.1);
            border-radius: 2px;
            margin-top: 10px;
            overflow: hidden;
        }
        
        .progress-fill {
            height: 100%;
            background: linear-gradient(135deg, rgba(76, 114, 89, 0.8) 0%, rgba(93, 138, 107, 0.8) 100%);
            width: 0%;
            transition: width 2s ease;
        }
        
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(10px); }
            to { opacity: 1; transform: translateY(0); }
        }
        
        @keyframes typing {
            0%, 60%, 100% { transform: scale(1); opacity: 0.5; }
            30% { transform: scale(1.3); opacity: 1; }
        }
        
        @media (max-width: 600px) {
            .chat-container {
                height: 100vh;
                border-radius: 0;
                max-width: 100%;
            }
            
            .message {
                max-width: 90%;
            }
            
            .header {
                padding: 20px;
            }
            
            .header h1 {
                font-size: 24px;
                letter-spacing: 3px;
            }
            
            .product-card {
                min-width: 240px;
            }
        }
    </style>
</head>
<body>
    <div class="chat-container">
        <div class="header">
            <div class="logo">
                <img src="https://i.ibb.co/QDTzVTY/image.png" alt="INGREVIA Logo" onerror="this.style.display='none'; this.nextElementSibling.style.display='block';" />
                <div class="logo-fallback" style="display: none;">
                    <svg viewBox="0 0 100 100" xmlns="http://www.w3.org/2000/svg">
                        <circle cx="50" cy="50" r="45" fill="none" stroke="rgba(76, 114, 89, 0.2)" stroke-width="2"/>
                        <circle cx="50" cy="50" r="35" fill="none" stroke="rgba(76, 114, 89, 0.3)" stroke-width="1"/>
                        
                        <path d="M50 25 Q40 35 35 45 Q40 50 50 45 Q60 50 65 45 Q60 35 50 25 Z" 
                              fill="rgba(76, 114, 89, 0.8)" stroke="rgba(76, 114, 89, 0.9)" stroke-width="0.5"/>
                        
                        <path d="M50 45 Q42 52 38 62 Q42 67 50 62 Q58 67 62 62 Q58 52 50 45 Z" 
                              fill="rgba(93, 138, 107, 0.7)" stroke="rgba(93, 138, 107, 0.8)" stroke-width="0.5"/>
                        
                        <path d="M50 62 Q44 68 41 76 Q44 80 50 76 Q56 80 59 76 Q56 68 50 62 Z" 
                              fill="rgba(76, 114, 89, 0.6)" stroke="rgba(76, 114, 89, 0.7)" stroke-width="0.5"/>
                        
                        <line x1="50" y1="45" x2="50" y2="85" stroke="rgba(76, 114, 89, 0.8)" stroke-width="2" stroke-linecap="round"/>
                        
                        <text x="50" y="20" text-anchor="middle" font-family="serif" font-size="6" fill="rgba(76, 114, 89, 0.9)" font-weight="300">INGREVIA</text>
                        <text x="50" y="92" text-anchor="middle" font-family="serif" font-size="4" fill="rgba(76, 114, 89, 0.7)" font-weight="300">INGREDIENT VIA</text>
                    </svg>
                </div>
            </div>
            <h1>INGREVIA</h1>
            <p>성분 기반 맞춤형 화장품 추천 에이전트</p>
        </div>
        
        <div class="chat-messages" id="chatMessages">
            <div class="welcome-message">
                <h2>안녕하세요! 🌿</h2>
                <p>당신의 피부 타입과 고민에 맞는<br>성분 기반 맞춤 화장품을 찾아드릴게요.</p>
                
                <div class="quick-start">
                    <h3>🎯 맞춤 추천을 시작해보세요</h3>
                    
                    <div class="selection-panel current" data-step="1">
                        <div class="selection-title">1. 찾고 계신 제품 카테고리를 선택해주세요</div>
                        <div class="button-group">
                            <button class="selection-btn" data-type="category" data-value="클렌징 폼">클렌징 폼</button>
                            <button class="selection-btn" data-type="category" data-value="스킨/토너">스킨/토너</button>
                            <button class="selection-btn" data-type="category" data-value="에센스/앰플/세럼">에센스/앰플/세럼</button>
                            <button class="selection-btn" data-type="category" data-value="로션/에멀전">로션/에멀전</button>
                            <button class="selection-btn" data-type="category" data-value="크림">크림</button>
                            <button class="selection-btn" data-type="category" data-value="선크림/로션">선크림/로션</button>
                            <button class="selection-btn" data-type="category" data-value="시트마스크">시트마스크</button>
                            <button class="selection-btn" data-type="category" data-value="밤/멀티밤">밤/멀티밤</button>
                        </div>
                    </div>
                    
                    <div class="selection-panel" data-step="2" style="opacity: 0.5;">
                        <div class="selection-title">2. 피부 타입을 선택해주세요</div>
                        <div class="button-group">
                            <button class="selection-btn" data-type="skin-type" data-value="민감성">민감성</button>
                            <button class="selection-btn" data-type="skin-type" data-value="지성">지성</button>
                            <button class="selection-btn" data-type="skin-type" data-value="건성">건성</button>
                            <button class="selection-btn" data-type="skin-type" data-value="아토피 피부">아토피 피부</button>
                            <button class="selection-btn" data-type="skin-type" data-value="미정(해당사항 없음)">미정</button>
                        </div>
                    </div>
                    
                    <div class="selection-panel" data-step="3" style="opacity: 0.5;">
                        <div class="selection-title">3. 피부 고민을 선택해주세요 (복수 선택 가능)</div>
                        <div class="button-group">
                            <button class="selection-btn" data-type="concern" data-value="진정">진정</button>
                            <button class="selection-btn" data-type="concern" data-value="미백">미백</button>
                            <button class="selection-btn" data-type="concern" data-value="보습">보습</button>
                            <button class="selection-btn" data-type="concern" data-value="주름/탄력">주름/탄력</button>
                            <button class="selection-btn" data-type="concern" data-value="모공/피지">모공/피지</button>
                            <button class="selection-btn" data-type="concern" data-value="없음">특별한 고민 없음</button>
                        </div>
                    </div>
                </div>
            </div>
            
            <div class="typing-indicator" id="typingIndicator">
                <span>INGREVIA가 성분을 분석중</span>
                <div class="typing-dots">
                    <div class="typing-dot"></div>
                    <div class="typing-dot"></div>
                    <div class="typing-dot"></div>
                </div>
            </div>
            
            <div class="progress-indicator" id="progressIndicator">
                <div>성분 데이터베이스에서 최적의 제품을 찾는 중...</div>
                <div class="progress-bar">
                    <div class="progress-fill" id="progressFill"></div>
                </div>
            </div>
        </div>
        
        <div class="input-container">
            <div class="input-wrapper">
                <textarea 
                    id="chatInput" 
                    class="chat-input" 
                    placeholder="자유롭게 질문해보세요... (예: 겨울용 보습크림 추천해줘, 민감성 피부에 좋은 세럼 찾아줘)"
                    rows="1"
                ></textarea>
                <button class="send-btn" id="sendBtn">
                    <svg width="22" height="22" fill="currentColor" viewBox="0 0 24 24">
                        <path d="M2 21L23 12 2 3V10L17 12 2 14V21Z"/>
                    </svg>
                </button>
            </div>
        </div>
    </div>

    <script>
        const chatMessages = document.getElementById('chatMessages');
        const chatInput = document.getElementById('chatInput');
        const sendBtn = document.getElementById('sendBtn');
        const typingIndicator = document.getElementById('typingIndicator');
        const progressIndicator = document.getElementById('progressIndicator');
        const progressFill = document.getElementById('progressFill');
        
        let userProfile = {
            category: '',
            skinType: '',
            concerns: []
        };
        
        let conversationHistory = [];
        let currentStep = 1;
        
        // 성분 기반 점수 계산을 위한 데이터
        const EFFICACY_INGREDIENTS = {
            '진정': ['병풀추출물', '마데카소사이드', '판테놀', '알란토인', '약모밀추출물'],
            '미백': ['나이아신아마이드', '알부틴', '비타민C', '글루타티온'],
            '보습': ['소듐하이알루로네이트', '글리세린', '세라마이드엔피', '스쿠알란', '베타-글루칸'],
            '주름/탄력': ['아데노신', '아세틸헥사펩타이드-8', '카퍼트라이펩타이드-1', '팔미토일펜타펩타이드-4', '하이드롤라이즈드콜라겐'],
            '모공/피지': ['살리실릭애씨드', '티트리잎오일', '버지니아풍년화추출물', '나이아신아마이드', '카프릴로일살리실릭애씨드']
        };

        const CAUTION_INGREDIENTS = {
            '민감성': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올'],
            '지성': ['미네랄오일', '시어버터', '코코넛야자일', '다이메티콘'],
            '건성': ['에탄올', '변성알코올', '살리실릭애씨드'],
            '아토피 피부': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올', '프로필렌글라이콜']
        };

        // EWG 등급 매핑 (간단화된 버전)
        const EWG_GRADES = {
            '정제수': 1, '글리세린': 1, '부틸렌글라이콜': 1, '나이아신아마이드': 1,
            '판테놀': 1, '알란토인': 1, '소듐하이알루로네이트': 1, '세라마이드엔피': 1,
            '병풀추출물': 1, '마데카소사이드': 1, '향료': 8, '리모넨': 4, '리날룰': 3
        };

        // 가상 제품 데이터베이스 (실제로는 서버에서 가져올 데이터)
        const PRODUCT_DATABASE = [
            {
                id: 1,
                name: "다이브인 저분자 히알루론산 세럼",
                brand: "토리든",
                category: "에센스/앰플/세럼",
                price: 22000,
                volume: "50ml",
                ingredients: "정제수;부틸렌글라이콜;글리세린;다이프로필렌글라이콜;1,2-헥산다이올;판테놀;소듐하이알루로네이트;하이드롤라이즈드하이알루로닉애씨드",
                link: "https://www.hwahae.co.kr/goods/라운드랩-1025-독도-토너-500ml/34633",
                image: "https://via.placeholder.com/80x80/5d8a6b/ffffff?text=토너"
            },
            {
                id: 3,
                name: "세라마이드 아토 로션",
                brand: "일리윤",
                category: "로션/에멀전",
                price: 26200,
                volume: "564ml",
                ingredients: "정제수;글리세린;프로판다이올;하이드로제네이티드쌀겨오일;세라마이드엔피;판테놀;알란토인",
                link: "https://www.hwahae.co.kr/goods/일리윤-세라마이드-아토-로션-350ml/58436",
                image: "https://via.placeholder.com/80x80/4c7259/ffffff?text=로션"
            },
            {
                id: 4,
                name: "아토베리어365 크림",
                brand: "에스트라",
                category: "크림",
                price: 33000,
                volume: "80ml",
                ingredients: "정제수;부틸렌글라이콜;글리세린;세라마이드엔피;나이아신아마이드;알란토인",
                link: "https://www.hwahae.co.kr/goods/에스트라-아토베리어365-크림-80ml/70590",
                image: "https://via.placeholder.com/80x80/5d8a6b/ffffff?text=크림"
            },
            {
                id: 5,
                name: "녹두 약산성 클렌징폼",
                brand: "비플레인",
                category: "클렌징 폼",
                price: 15000,
                volume: "80ml",
                ingredients: "글리세린;정제수;소듐코코일글라이시네이트;소듐라우로일글루타메이트;녹두가루;알란토인",
                link: "https://www.hwahae.co.kr/goods/비플레인-녹두-약산성-클렌징폼-200ml/67703",
                image: "https://via.placeholder.com/80x80/4c7259/ffffff?text=폼"
            },
            {
                id: 6,
                name: "자작나무 수분 선크림",
                brand: "라운드랩",
                category: "선크림/로션",
                price: 40000,
                volume: "80ml",
                ingredients: "정제수;다이부틸아디페이트;프로판다이올;자작나무수액;소듐하이알루로네이트;알란토인",
                link: "https://www.hwahae.co.kr/goods/라운드랩-자작나무-수분-선크림-80ml/60938",
                image: "https://via.placeholder.com/80x80/5d8a6b/ffffff?text=선크림"
            },
            {
                id: 7,
                name: "마데카소사이드 에센셜 마스크",
                brand: "메디힐",
                category: "시트마스크",
                price: 40000,
                volume: "24ml*20매",
                ingredients: "정제수;프로판다이올;나이아신아마이드;글리세린;마데카소사이드;병풀추출물;판테놀",
                link: "https://www.hwahae.co.kr/goods/메디힐-마데카소사이드-에센셜-마스크/64587",
                image: "https://via.placeholder.com/80x80/4c7259/ffffff?text=마스크"
            },
            {
                id: 8,
                name: "시카플라스트 밤 B5+",
                brand: "라로슈포제",
                category: "밤/멀티밤",
                price: 39000,
                volume: "100ml",
                ingredients: "정제수;부틸렌글라이콜;판테놀;스쿠알란;마데카소사이드;아시아티코사이드",
                link: "https://www.hwahae.co.kr/products/라로슈포제-시카플라스트-밤-B5PLUS/2010052",
                image: "https://via.placeholder.com/80x80/5d8a6b/ffffff?text=밤"
            }
        ];

        // 선택 버튼 이벤트
        document.addEventListener('click', function(e) {
            if (e.target.classList.contains('selection-btn')) {
                const type = e.target.dataset.type;
                const value = e.target.dataset.value;
                
                if (type === 'skin-type' || type === 'category') {
                    // 단일 선택
                    document.querySelectorAll(`[data-type="${type}"]`).forEach(btn => {
                        btn.classList.remove('selected');
                    });
                    e.target.classList.add('selected');
                    
                    if (type === 'skin-type') {
                        userProfile.skinType = value;
                        addToHistory('선택', `피부타입: ${value}`);
                        
                        // 피부 고민 단계로 진행
                        if (currentStep === 2) {
                            nextStep();
                        }
                        
                        // 필수 선택 완료 시 추천 시작
                        if (userProfile.category && userProfile.skinType) {
                            setTimeout(() => {
                                startRecommendation();
                            }, 800);
                        }
                    } else if (type === 'category') {
                        userProfile.category = value;
                        addToHistory('선택', `카테고리: ${value}`);
                        
                        // 피부타입 선택 단계로 진행
                        if (currentStep === 1) {
                            nextStep();
                        }
                    }
                } else if (type === 'concern') {
                    // 다중 선택
                    if (value === '없음') {
                        // '없음' 선택 시 다른 모든 선택 해제
                        document.querySelectorAll(`[data-type="${type}"]`).forEach(btn => {
                            btn.classList.remove('selected');
                        });
                        e.target.classList.add('selected');
                        userProfile.concerns = [];
                        addToHistory('선택', `피부고민: 특별한 고민 없음`);
                    } else {
                        // '없음' 해제하고 현재 선택 토글
                        const noneBtn = document.querySelector(`[data-type="${type}"][data-value="없음"]`);
                        if (noneBtn) noneBtn.classList.remove('selected');
                        
                        e.target.classList.toggle('selected');
                        
                        if (e.target.classList.contains('selected')) {
                            if (!userProfile.concerns.includes(value)) {
                                userProfile.concerns.push(value);
                                addToHistory('선택', `피부고민 추가: ${value}`);
                            }
                        } else {
                            userProfile.concerns = userProfile.concerns.filter(c => c !== value);
                            addToHistory('선택', `피부고민 제거: ${value}`);
                        }
                    }
                }
            }
        });

        function nextStep() {
            // 현재 단계 완료 표시
            const currentPanel = document.querySelector(`[data-step="${currentStep}"]`);
            if (currentPanel) {
                currentPanel.classList.remove('current');
                currentPanel.classList.add('completed');
            }

            // 다음 단계 활성화
            currentStep++;
            const nextPanel = document.querySelector(`[data-step="${currentStep}"]`);
            if (nextPanel) {
                nextPanel.style.opacity = '1';
                nextPanel.classList.add('current');
            }
        }

        function addToHistory(type, content) {
            conversationHistory.push({
                timestamp: new Date(),
                type: type,
                content: content
            });
        }

        function calculateProductScore(product, userConcerns, userSkinType) {
            let score = 50; // 기본 점수
            
            if (!product.ingredients) return score;
            
            const ingredients = product.ingredients.split(';');
            
            // EWG 안전성 점수
            let ewgScore = 0;
            ingredients.forEach(ingredient => {
                const grade = EWG_GRADES[ingredient] || 3;
                ewgScore += (11 - grade);
            });
            score += ewgScore / ingredients.length;
            
            // 효능 성분 점수
            if (userConcerns && userConcerns.length > 0) {
                userConcerns.forEach(concern => {
                    if (EFFICACY_INGREDIENTS[concern]) {
                        const matchCount = ingredients.filter(ing => 
                            EFFICACY_INGREDIENTS[concern].includes(ing)
                        ).length;
                        score += matchCount * 5;
                    }
                });
            }
            
            // 주의 성분 감점
            if (userSkinType && CAUTION_INGREDIENTS[userSkinType]) {
                const cautionCount = ingredients.filter(ing => 
                    CAUTION_INGREDIENTS[userSkinType].includes(ing)
                ).length;
                score -= cautionCount * 3;
            }
            
            return Math.max(0, score);
        }

        function startRecommendation() {
            // 웰컴 메시지 숨기기
            document.querySelector('.welcome-message').style.display = 'none';
            
            // 사용자 선택사항 메시지 추가
            let concernsText = userProfile.concerns.length > 0 ? userProfile.concerns.join(', ') : '특별한 고민 없음';
            const profileMessage = `선택 정보
• 카테고리: ${userProfile.category}
• 피부 타입: ${userProfile.skinType} 
• 피부 고민: ${concernsText}`;
            
            addMessage(profileMessage, 'user');
            addToHistory('프로필', profileMessage);
            
            // 분석 진행 시뮬레이션
            setTimeout(() => {
                showProgress();
                startProgressAnimation();
                
                setTimeout(() => {
                    hideProgress();
                    showTyping();
                    
                    setTimeout(() => {
                        hideTyping();
                        generateRecommendation();
                    }, 2000);
                }, 3000);
            }, 1000);
        }
        
        function generateRecommendation() {
            // 카테고리에 맞는 제품 필터링
            const categoryProducts = PRODUCT_DATABASE.filter(product => 
                product.category === userProfile.category
            );
            
            // 점수 계산 및 정렬
            const scoredProducts = categoryProducts.map(product => ({
                ...product,
                score: calculateProductScore(product, userProfile.concerns, userProfile.skinType)
            })).sort((a, b) => b.score - a.score);
            
            // 상위 5개 제품 선택
            const topProducts = scoredProducts.slice(0, 5);
            
            if (topProducts.length === 0) {
                addMessage(`죄송합니다. '${userProfile.category}' 카테고리에서 적합한 제품을 찾지 못했습니다. 다른 카테고리를 선택해보시겠어요?`, 'bot');
                return;
            }

            let responseText = `성분 분석을 통한 맞춤 추천이 완료되었어요!

**${userProfile.category}** 카테고리에서 **${userProfile.skinType}** 피부 타입에 최적화된 제품들을 찾았습니다.

**추천 근거:**
• EWG 안전성 등급을 기반으로 한 성분 분석
• 피부 타입별 주의 성분 필터링 적용
• 선택하신 피부 고민에 효과적인 성분 우선 추천

아래 제품들을 좌우로 스크롤하며 비교해보세요! 이미지를 클릭하면 구매 페이지로 이동합니다.`;

            addMessage(responseText, 'bot');
            
            // 제품 갤러리 추가
            addProductGallery(topProducts);
            
            addToHistory('추천완료', `${topProducts.length}개 제품 추천`);
        }

        function addProductGallery(products) {
            const galleryHtml = `
                <div class="product-gallery">
                    <div class="product-scroll">
                        ${products.map((product, index) => `
                            <div class="product-card" onclick="window.open('${product.link}', '_blank')">
                                <div class="product-image">
                                    <img src="${product.image}" alt="${product.name}" />
                                </div>
                                <div class="product-info">
                                    <div class="product-rank">${index + 1}위 추천</div>
                                    <div class="product-name">${product.name}</div>
                                    <div class="product-brand">${product.brand}</div>
                                    <div class="product-price">${product.price.toLocaleString()}원</div>
                                    <div class="product-volume">${product.volume}</div>
                                </div>
                            </div>
                        `).join('')}
                    </div>
                </div>
            `;
            
            const messageDiv = document.createElement('div');
            messageDiv.classList.add('message', 'bot');
            messageDiv.innerHTML = galleryHtml;
            chatMessages.appendChild(messageDiv);
            chatMessages.scrollTop = chatMessages.scrollHeight;
        }
        
        function showProgress() {
            progressIndicator.style.display = 'block';
            chatMessages.scrollTop = chatMessages.scrollHeight;
        }
        
        function hideProgress() {
            progressIndicator.style.display = 'none';
        }
        
        function startProgressAnimation() {
            progressFill.style.width = '0%';
            setTimeout(() => {
                progressFill.style.width = '100%';
            }, 100);
        }
        
        function addMessage(text, sender) {
            const messageDiv = document.createElement('div');
            messageDiv.classList.add('message', sender);
            
            // 마크다운 형식의 텍스트를 HTML로 변환 (간단 버전)
            let htmlText = text
                .replace(/\*\*(.*?)\*\*/g, '<strong>$1</strong>')
                .replace(/\n/g, '<br>')
                .replace(/• /g, '&bull; ');
            
            messageDiv.innerHTML = htmlText;
            chatMessages.appendChild(messageDiv);
            chatMessages.scrollTop = chatMessages.scrollHeight;
            
            addToHistory('메시지', text);
        }
        
        function showTyping() {
            typingIndicator.style.display = 'flex';
            chatMessages.scrollTop = chatMessages.scrollHeight;
        }
        
        function hideTyping() {
            typingIndicator.style.display = 'none';
        }

        // 채팅 입력 기능
        function handleSendMessage() {
            const message = chatInput.value.trim();
            if (message) {
                addMessage(message, 'user');
                chatInput.value = '';
                
                // 간단한 챗봇 응답
                setTimeout(() => {
                    showTyping();
                    setTimeout(() => {
                        hideTyping();
                        
                        let response = "죄송합니다. 현재는 선택 버튼을 통한 맞춤 추천 기능만 지원합니다. 위의 선택지를 통해 카테고리, 피부타입, 고민을 선택해주세요!";
                        
                        // 키워드 기반 간단 응답
                        if (message.includes('추천') || message.includes('제품')) {
                            response = "맞춤 추천을 받으시려면 위의 선택 버튼을 통해 다음 정보를 선택해주세요:\n1. 제품 카테고리\n2. 피부 타입\n3. 피부 고민\n\n선택이 완료되면 자동으로 성분 분석을 통한 개인화된 추천을 제공해드립니다!";
                        }
                        
                        addMessage(response, 'bot');
                    }, 1000);
                }, 500);
            }
        }

        sendBtn.addEventListener('click', handleSendMessage);
        
        chatInput.addEventListener('keypress', function(e) {
            if (e.key === 'Enter' && !e.shiftKey) {
                e.preventDefault();
                handleSendMessage();
            }
        });

        // 텍스트 영역 자동 높이 조절
        chatInput.addEventListener('input', function() {
            this.style.height = 'auto';
            this.style.height = Math.min(this.scrollHeight, 120) + 'px';
        });
    </script>
</body>
</html>www.hwahae.co.kr/goods/토리든-다이브인-저분자-히알루론산-세럼-50ml/54413",
                image: "https://via.placeholder.com/80x80/4c7259/ffffff?text=세럼"
            },
            {
                id: 2,
                name: "1025 독도 토너",
                brand: "라운드랩",
                category: "스킨/토너",
                price: 15000,
                volume: "200ml",
                ingredients: "정제수;부틸렌글라이콜;글리세린;펜틸렌글라이콜;프로판다이올;아이리쉬모스추출물;사탕수수추출물;해수",
                link: "https://

In [ ]:
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>INGREVIA - AI 화장품 추천 챗봇</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Noto Sans KR', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
            background: linear-gradient(135deg, #e8e8e8 0%, #f5f5f5 100%);
            min-height: 100vh;
            display: flex;
            justify-content: center;
            align-items: center;
            padding: 20px;
        }

        .chat-container {
            width: 100%;
            max-width: 800px;
            height: 85vh;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.1);
            display: flex;
            flex-direction: column;
            overflow: hidden;
        }

        .chat-header {
            background: linear-gradient(135deg, #2d5a27 0%, #4a7c47 100%);
            color: white;
            padding: 20px;
            text-align: center;
            position: relative;
        }

        .logo-wrapper {
            display: inline-block;
            background: #e0e4d7;
            border-radius: 50%;
            padding: 15px;
            margin-bottom: 10px;
        }

        .logo-svg {
            width: 80px;
            height: 80px;
            display: block;
        }

        .brand-title {
            font-size: 2rem;
            font-weight: 300;
            letter-spacing: 3px;
            margin-bottom: 5px;
        }

        .brand-subtitle {
            font-size: 0.9rem;
            opacity: 0.9;
        }

        .chat-messages {
            flex: 1;
            overflow-y: auto;
            padding: 20px;
            background: #fafafa;
        }

        .message {
            margin-bottom: 15px;
            display: flex;
            align-items: flex-start;
            animation: fadeIn 0.3s ease;
        }

        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(10px); }
            to { opacity: 1; transform: translateY(0); }
        }

        .message.user {
            justify-content: flex-end;
        }

        .message-content {
            max-width: 70%;
            padding: 12px 16px;
            border-radius: 18px;
            line-height: 1.5;
        }

        .bot .message-content {
            background: white;
            border: 1px solid #e0e0e0;
            border-bottom-left-radius: 4px;
            margin-left: 10px;
        }

        .user .message-content {
            background: #2d5a27;
            color: white;
            border-bottom-right-radius: 4px;
            margin-right: 10px;
        }

        .quick-replies {
            display: flex;
            flex-wrap: wrap;
            gap: 8px;
            margin-top: 10px;
        }

        .quick-reply-btn {
            background: white;
            border: 2px solid #4a7c47;
            color: #2d5a27;
            padding: 8px 16px;
            border-radius: 20px;
            cursor: pointer;
            transition: all 0.2s;
            font-size: 0.95rem;
        }

        .quick-reply-btn:hover {
            background: #4a7c47;
            color: white;
        }

        .quick-reply-btn.selected {
            background: #2d5a27;
            color: white;
            border-color: #2d5a27;
        }

        .selection-area {
            margin-top: 10px;
            padding: 10px;
            background: #f0f4f0;
            border-radius: 10px;
        }

        .complete-btn {
            background: #2d5a27;
            color: white;
            border: none;
            padding: 10px 20px;
            border-radius: 20px;
            cursor: pointer;
            margin-top: 10px;
            font-weight: bold;
            display: none;
        }

        .complete-btn.show {
            display: inline-block;
        }

        .product-card {
            background: white;
            border: 1px solid #e0e0e0;
            border-radius: 12px;
            padding: 15px;
            margin: 10px 0;
            cursor: pointer;
            transition: all 0.2s;
        }

        .product-card:hover {
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
            transform: translateY(-2px);
        }

        .product-rank {
            display: inline-block;
            background: #ffd700;
            color: #2d5a27;
            padding: 4px 10px;
            border-radius: 15px;
            font-weight: bold;
            font-size: 0.9rem;
        }

        .product-name {
            font-weight: bold;
            color: #2d5a27;
            margin: 8px 0;
        }

        .product-detail {
            font-size: 0.9rem;
            color: #666;
            margin: 4px 0;
        }

        .loading {
            text-align: center;
            padding: 20px;
        }

        .loading-dots {
            display: inline-flex;
            gap: 4px;
        }

        .dot {
            width: 8px;
            height: 8px;
            background: #4a7c47;
            border-radius: 50%;
            animation: bounce 1.4s infinite ease-in-out;
        }

        .dot:nth-child(1) { animation-delay: -0.32s; }
        .dot:nth-child(2) { animation-delay: -0.16s; }

        @keyframes bounce {
            0%, 80%, 100% { transform: scale(0); }
            40% { transform: scale(1); }
        }

        .chat-input {
            padding: 20px;
            background: white;
            border-top: 1px solid #e0e0e0;
            display: flex;
            gap: 10px;
        }

        .input-field {
            flex: 1;
            padding: 10px 15px;
            border: 2px solid #e0e0e0;
            border-radius: 25px;
            outline: none;
        }

        .input-field:focus {
            border-color: #4a7c47;
        }

        .send-btn {
            background: #2d5a27;
            color: white;
            border: none;
            padding: 10px 20px;
            border-radius: 25px;
            cursor: pointer;
        }

        .send-btn:hover {
            background: #4a7c47;
        }

        .send-btn:disabled {
            background: #ccc;
            cursor: not-allowed;
        }
    </style>
</head>
<body>
    <div class="chat-container">
        <div class="chat-header">
            <div class="logo-wrapper">
                <svg class="logo-svg" viewBox="0 0 200 200" xmlns="http://www.w3.org/2000/svg">
                    <!-- 외부 원 -->
                    <circle cx="100" cy="100" r="95" fill="none" stroke="#2d5a27" stroke-width="4"/>
                    <circle cx="100" cy="100" r="85" fill="none" stroke="#2d5a27" stroke-width="2"/>
                    
                    <!-- INGREVIA 텍스트 (상단 곡선) -->
                    <path id="topPath" d="M 30,100 A 70,70 0 0,1 170,100" fill="none"/>
                    <text font-family="serif" font-size="24" font-weight="500" fill="#2d5a27">
                        <textPath href="#topPath" startOffset="50%" text-anchor="middle">INGREVIA</textPath>
                    </text>
                    
                    <!-- 중앙 식물 디자인 -->
                    <g transform="translate(100, 100)">
                        <!-- 줄기 -->
                        <line x1="0" y1="-20" x2="0" y2="30" stroke="#2d5a27" stroke-width="3"/>
                        
                        <!-- 잎사귀들 -->
                        <ellipse cx="-15" cy="-10" rx="12" ry="6" fill="none" stroke="#2d5a27" stroke-width="2" transform="rotate(-30 -15 -10)"/>
                        <ellipse cx="15" cy="-10" rx="12" ry="6" fill="none" stroke="#2d5a27" stroke-width="2" transform="rotate(30 15 -10)"/>
                        <ellipse cx="-18" cy="5" rx="14" ry="7" fill="none" stroke="#2d5a27" stroke-width="2" transform="rotate(-25 -18 5)"/>
                        <ellipse cx="18" cy="5" rx="14" ry="7" fill="none" stroke="#2d5a27" stroke-width="2" transform="rotate(25 18 5)"/>
                        <ellipse cx="-15" cy="20" rx="12" ry="6" fill="none" stroke="#2d5a27" stroke-width="2" transform="rotate(-35 -15 20)"/>
                        <ellipse cx="15" cy="20" rx="12" ry="6" fill="none" stroke="#2d5a27" stroke-width="2" transform="rotate(35 15 20)"/>
                    </g>
                    
                    <!-- INGREDIENT VIA 텍스트 (하단 곡선) -->
                    <path id="bottomPath" d="M 30,100 A 70,70 0 0,0 170,100" fill="none"/>
                    <text font-family="serif" font-size="16" fill="#2d5a27">
                        <textPath href="#bottomPath" startOffset="50%" text-anchor="middle">INGREDIENT VIA</textPath>
                    </text>
                </svg>
            </div>
            <h1 class="brand-title">INGREVIA</h1>
            <p class="brand-subtitle">AI 맞춤 화장품 추천 서비스</p>
        </div>

        <div class="chat-messages" id="chatMessages"></div>

        <div class="chat-input">
            <input type="text" class="input-field" id="userInput" placeholder="메시지를 입력하세요..." disabled>
            <button class="send-btn" id="sendBtn" disabled>전송</button>
        </div>
    </div>

    <script>
        // 챗봇 상태 관리
        const botState = {
            currentStep: 'init',
            skinType: null,
            concerns: [],
            selectedConcerns: new Set(),
            category: null
        };

        // CSV 데이터 시뮬레이션
        const mockProducts = [
            {제품명: "워터뱅크 블루 히알루론 크림", 브랜드명: "라네즈", 카테고리: "크림", 효능: "보습 진정", 전성분: "히알루론산,세라마이드,판테놀,나이아신아마이드", 가격: "39000", 용량: "50ml", 링크: "https://www.laneige.com", 유해성_점수: "1.8"},
            {제품명: "진정 리페어 크림", 브랜드명: "닥터자르트", 카테고리: "크림", 효능: "진정 보습", 전성분: "센텔라아시아티카,마데카소사이드,알란토인,글리세린", 가격: "48000", 용량: "50ml", 링크: "https://www.drjart.com", 유해성_점수: "1.5"},
            {제품명: "블랙스네일 크림", 브랜드명: "미샤", 카테고리: "크림", 효능: "주름/탄력 보습", 전성분: "달팽이점액여과물,아데노신,나이아신아마이드,펩타이드", 가격: "32000", 용량: "50ml", 링크: "https://www.missha.com", 유해성_점수: "2.1"},
            {제품명: "제로 세범 토너", 브랜드명: "이니스프리", 카테고리: "스킨/토너", 효능: "모공/피지 진정", 전성분: "녹차추출물,살리실산,나이아신아마이드,위치하젤", 가격: "18000", 용량: "200ml", 링크: "https://www.innisfree.com", 유해성_점수: "1.9"},
            {제품명: "화이트닝 에센스", 브랜드명: "설화수", 카테고리: "에센스/앰플/세럼", 효능: "미백 보습", 전성분: "나이아신아마이드,알부틴,감초추출물,인삼추출물", 가격: "120000", 용량: "50ml", 링크: "https://www.sulwhasoo.com", 유해성_점수: "2.5"}
        ];

        // 메시지 추가 함수
        function addMessage(content, isUser = false) {
            const messagesDiv = document.getElementById('chatMessages');
            const messageDiv = document.createElement('div');
            messageDiv.className = `message ${isUser ? 'user' : 'bot'}`;
            
            const contentDiv = document.createElement('div');
            contentDiv.className = 'message-content';
            contentDiv.innerHTML = content;
            
            messageDiv.appendChild(contentDiv);
            messagesDiv.appendChild(messageDiv);
            messagesDiv.scrollTop = messagesDiv.scrollHeight;
            
            return contentDiv;
        }

        // Quick Reply 버튼 추가
        function addQuickReplies(replies, multiSelect = false) {
            const messagesDiv = document.getElementById('chatMessages');
            const replyContainer = document.createElement('div');
            replyContainer.className = 'quick-replies';
            
            replies.forEach(reply => {
                const btn = document.createElement('button');
                btn.className = 'quick-reply-btn';
                btn.textContent = reply;
                btn.onclick = () => {
                    if (multiSelect) {
                        handleMultiSelect(btn, reply);
                    } else {
                        handleQuickReply(reply);
                    }
                };
                replyContainer.appendChild(btn);
            });
            
            messagesDiv.appendChild(replyContainer);
            
            if (multiSelect) {
                const selectionArea = document.createElement('div');
                selectionArea.className = 'selection-area';
                selectionArea.innerHTML = '<div id="selectedItems">선택된 항목: 없음</div>';
                
                const completeBtn = document.createElement('button');
                completeBtn.className = 'complete-btn';
                completeBtn.textContent = '선택 완료';
                completeBtn.id = 'completeBtn';
                completeBtn.onclick = handleCompleteSelection;
                
                selectionArea.appendChild(completeBtn);
                messagesDiv.appendChild(selectionArea);
            }
            
            messagesDiv.scrollTop = messagesDiv.scrollHeight;
        }

        // 다중 선택 처리
        function handleMultiSelect(btn, value) {
            if (botState.selectedConcerns.has(value)) {
                botState.selectedConcerns.delete(value);
                btn.classList.remove('selected');
            } else {
                if (value === '해당없음') {
                    // 해당없음 선택시 다른 모든 선택 취소
                    document.querySelectorAll('.quick-reply-btn').forEach(b => b.classList.remove('selected'));
                    botState.selectedConcerns.clear();
                }
                botState.selectedConcerns.add(value);
                btn.classList.add('selected');
            }
            
            updateSelectedItems();
        }

        // 선택 항목 업데이트
        function updateSelectedItems() {
            const selectedDiv = document.getElementById('selectedItems');
            const completeBtn = document.getElementById('completeBtn');
            
            if (botState.selectedConcerns.size > 0) {
                const items = Array.from(botState.selectedConcerns);
                selectedDiv.textContent = `선택된 항목: ${items.join(', ')}`;
                completeBtn.classList.add('show');
            } else {
                selectedDiv.textContent = '선택된 항목: 없음';
                completeBtn.classList.remove('show');
            }
        }

        // 선택 완료 처리
        function handleCompleteSelection() {
            if (botState.selectedConcerns.size === 0) {
                alert('최소 하나 이상 선택해주세요.');
                return;
            }
            
            botState.concerns = Array.from(botState.selectedConcerns);
            const selected = botState.concerns.join(', ');
            addMessage(selected, true);
            
            // Quick reply 버튼들 제거
            document.querySelectorAll('.quick-replies, .selection-area').forEach(el => el.remove());
            
            setTimeout(() => {
                askCategory();
            }, 500);
        }

        // Quick Reply 처리
        function handleQuickReply(reply) {
            addMessage(reply, true);
            
            // Quick reply 버튼들 제거
            document.querySelectorAll('.quick-replies').forEach(el => el.remove());
            
            setTimeout(() => {
                processUserInput(reply);
            }, 500);
        }

        // 사용자 입력 처리
        function processUserInput(input) {
            switch(botState.currentStep) {
                case 'init':
                    if (input.includes('시작')) {
                        askSkinType();
                    }
                    break;
                case 'skinType':
                    botState.skinType = input === '해당없음' ? null : input;
                    askConcerns();
                    break;
                case 'category':
                    botState.category = input;
                    showAnalysis();
                    break;
                case 'complete':
                    if (input.includes('다시')) {
                        resetBot();
                    }
                    break;
            }
        }

        // 피부 타입 질문
        function askSkinType() {
            botState.currentStep = 'skinType';
            addMessage('피부 타입을 선택해주세요:');
            addQuickReplies(['민감성', '지성', '건성', '아토피성', '해당없음']);
        }

        // 피부 고민 질문
        function askConcerns() {
            botState.currentStep = 'concerns';
            addMessage('피부 고민을 모두 선택해주세요: (복수 선택 가능)');
            addQuickReplies(['보습', '진정', '미백', '주름/탄력', '모공/피지', '해당없음'], true);
        }

        // 제품 카테고리 질문
        function askCategory() {
            botState.currentStep = 'category';
            addMessage('원하시는 제품 종류를 선택해주세요:');
            addQuickReplies(['스킨/토너', '로션/에멀전', '에센스/앰플/세럼', '크림', '클렌징 폼', '시트마스크']);
        }

        // 분석 및 추천
        function showAnalysis() {
            addMessage(`<div class="loading">
                <p>AI가 분석 중입니다...</p>
                <div class="loading-dots">
                    <div class="dot"></div>
                    <div class="dot"></div>
                    <div class="dot"></div>
                </div>
            </div>`);
            
            setTimeout(() => {
                showRecommendations();
            }, 2000);
        }

        // 추천 결과 표시
        function showRecommendations() {
            // 카테고리로 필터링
            let filtered = mockProducts.filter(p => p.카테고리 === botState.category);
            
            // 고민으로 필터링
            if (botState.concerns.length > 0 && !botState.concerns.includes('해당없음')) {
                filtered = filtered.filter(p => {
                    return botState.concerns.some(concern => p.효능.includes(concern));
                });
            }
            
            // 상위 3개 선택
            const top3 = filtered.slice(0, 3);
            
            if (top3.length === 0) {
                addMessage('죄송합니다. 조건에 맞는 제품을 찾지 못했습니다.');
                addQuickReplies(['다시 추천받기']);
                botState.currentStep = 'complete';
                return;
            }
            
            let resultHTML = '<div><h3>🎯 맞춤 추천 TOP 3</h3>';
            
            top3.forEach((product, index) => {
                resultHTML += `
                    <div class="product-card" onclick="window.open('${product.링크}', '_blank')">
                        <span class="product-rank">TOP ${index + 1}</span>
                        <div class="product-name">${product.제품명}</div>
                        <div class="product-detail">브랜드: ${product.브랜드명}</div>
                        <div class="product-detail">주요 성분: ${product.전성분}</div>
                        <div class="product-detail">가격: ${parseInt(product.가격).toLocaleString()}원 / ${product.용량}</div>
                        <div class="product-detail">안전도: ${product.유해성_점수 < 2 ? '우수' : '보통'} (${product.유해성_점수})</div>
                    </div>
                `;
            });
            
            resultHTML += '<p style="margin-top: 15px; font-size: 0.9rem; color: #666;">제품을 클릭하면 상세 페이지로 이동합니다.</p></div>';
            
            addMessage(resultHTML);
            addQuickReplies(['다시 추천받기']);
            botState.currentStep = 'complete';
        }

        // 봇 초기화
        function resetBot() {
            botState.currentStep = 'init';
            botState.skinType = null;
            botState.concerns = [];
            botState.selectedConcerns.clear();
            botState.category = null;
            
            document.getElementById('chatMessages').innerHTML = '';
            initBot();
        }

        // 봇 시작
        function initBot() {
            addMessage('안녕하세요! INGREVIA 맞춤 화장품 추천 서비스입니다.<br>당신의 피부에 딱 맞는 제품을 찾아드릴게요.');
            addQuickReplies(['시작하기']);
        }

        // 페이지 로드시 실행
        window.onload = () => {
            initBot();
        };
    </script>
</body>
</html>